### Landmark to feature : Top

In [1]:
from feature_process import *
import numpy as np
import os

In [2]:
def get_name(path):
    sp = path.split('/')
    return sp[-1]

    # name = sp[-2]
    # if sp[-1].find('basal')!=-1:
    #     bt = 'basal'
    # else:
    #     bt = 'treat'
    # return name+'-'+bt+'.csv'

In [3]:
# get csv paths
paths  = []
for root,dirs,files in os.walk(r'C:\Users\x\Desktop\vid_top\csv'):
    root = root.replace('\\','/')
    if(root.find('')!=-1): #select mice
        for f in files:
            if(f.find('.csv')!=-1): #select file 
                paths.append(root+'/'+f)

In [4]:
save_root = r'C:\Users\x\Desktop\vid_top\feat/'

# select feature ###########################
sel_dist = []
for i in range(6):
    for j in range(i+1,7):
        sel_dist.append([i,j])
sel_ang = []
# for i in range(5):
# i=0
# for j in range(i+1,6):
#     for k in range(j+1,7):
#         sel_ang.append([i,j,k])
            # sel_ang.append([i,k,j])
            # sel_ang.append([j,i,k])
############################################

if not os.path.isdir(save_root):
    os.makedirs(save_root)
for path in paths:
    dlc1 = dlc(path, True)
    ## feature transfer
    feat = combine_feat(dlc1, sel_dist, sel_ang)
    ## copy and rename
    # feat = dlc1.raw
    fname = get_name(path)
    np.savetxt(save_root+fname,feat,delimiter=",", fmt='%s')

In [3]:
# select all features
sel_dist = []
for i in range(6):
    for j in range(i+1,7):
        sel_dist.append([i,j])
sel_ang = []
# for i in range(5):
i=0
for j in range(i+1,6):
    for k in range(j+1,7):
        sel_ang.append([i,j,k])

# sel_dist = [[0,3],[3,6],[1,3],[2,3]]
# sel_ang = [[0,3,6],[1,4,6],[2,5,6]]

In [5]:
# single process
save_file = r'C:\Users\x\Desktop\vid_top\0test\short_feat.csv'
path = r'C:\Users\x\Desktop\vid_top\0test\short.csv'
dlc1 = dlc(path, True)
feat = combine_feat(dlc1, sel_dist, sel_ang)
np.savetxt(save_file,feat,delimiter=",", fmt='%s')

### Landmark to feature : Front

In [1]:
from feature_process_front import *
import numpy as np
import os

In [3]:
inpath = r'C:\Users\x\Desktop\front\exp1_xml\0'
save_path = r'C:\Users\x\Desktop\front\exp1_feat0'
postfix = '-treat.csv'
pythonenv = r'C:\Users\x\.conda\envs\micelab\python.exe'

if not os.path.isdir(save_path):
    os.makedirs(save_path)

for root,dirs,files in os.walk(inpath):
    root = root.replace('\\','/')
    if(root.find('')!=-1): #select mice
        for f in files:
            if(f.find('.xml')!=-1): #select file 
                xmlpath = root+'/'+f
                csvpath = save_path +'/'+ f.replace('.xml', postfix)
                os.system(pythonenv+' feature_process_front.py --inpath '+xmlpath+' --outpath '+csvpath)
                # print(pythonenv+' feature_process_front.py --inpath '+xmlpath+' --outpath '+csvpath)

### Cluster Output (Load Mat file)

In [5]:
import scipy.io
import numpy as np
import cv2
import imageio

In [6]:
seg = scipy.io.loadmat('./mat/seg.mat')['seg'].T
label = scipy.io.loadmat('./mat/label.mat')['label'].T

In [4]:
def seg_gif(vid_path, seg, label,out_path='./result'):
    '''
    save video clips gif to out_path by classes
    seg : segment points
    label : one-hot label
    '''
    cap = cv2.VideoCapture(vid_path)

    for i in range(len(label[0])):
        cpath = out_path+'/'+str(i)+'/'
        if not os.path.isdir(cpath):
            os.makedirs(cpath)

    vid_i=0
    imgs = []
    segi=1
    i=0
    while(cap.isOpened()):
        ret, frame = cap.read()
        if not ret:
            break
        # frame in clip
        if i<seg[segi]:
            imgs.append(frame)
        # split point reached
        if i == seg[segi]:
            cpath = out_path+'/'+str(np.argmax(label[segi-1]))+'/'
            imageio.mimsave(cpath+"/"+str(vid_i)+".gif",imgs)
            imgs = []
            imgs.append(frame)
            vid_i += 1
            segi += 1
        i+=1
    cap.release()

In [5]:
seg_gif('./F2.avi',seg,label)